In [1]:
import random
from datetime import datetime, timedelta
from IPython.display import display, HTML
import ipywidgets as widgets

# بيانات الفرق والملاعب والأوقات
teams = [
    "Al Ahly", "Zamalek", "Pyramids", "Masry", "Future", "Ismaily",
    "Smouha", "ENPPI", "Ceramica", "National Bank", "Talaea El Gaish",
    "Alexandria Union", "El Dakhleya", "El Gouna", "Zed",
    "Modern Sport", "Pharco", "Wadi Degla"
]

venues = [
    "Cairo Stadium", "Borg El Arab", "Air Defense Stadium",
    "Suez Stadium", "Alexandria Stadium", "Petro Sport Stadium",
    "Military Academy Stadium", "Al Salam Stadium", "El Sekka El Hadeed Stadium",
    "Zed Club Stadium"
]

match_times = ["17:00", "20:00"]

In [2]:
# مدة البطولة
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)

# إعداد أيام البطولة
all_dates = []
current_date = start_date
while current_date <= end_date:
    all_dates.append(current_date)
    current_date += timedelta(days=1)

# تعريف الـ Match
class Match:
    def __init__(self, team1, team2, date, time, venue):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue

    def __repr__(self):
        return f"{self.team1} vs {self.team2} on {self.date.strftime('%Y-%m-%d')} at {self.time} in {self.venue}"

# توليد جدول دوري أسبوعي مع تكرار المباريات مرتين
def generate_weekly_schedule(teams, venues, all_dates, match_times, min_rest_days=4):
    schedule = []
    last_played = {team: start_date - timedelta(days=min_rest_days) for team in teams}
    day_info = {d: {'count': 0, 'venues': set(), 'times': set()} for d in all_dates}
    pairings = [(t1, t2) for t1 in teams for t2 in teams if t1 != t2]
    random.shuffle(pairings)
    matches_count = {(t1, t2): 0 for t1 in teams for t2 in teams if t1 != t2}
    week_dates = all_dates[::7]

    for week_start in week_dates:
        used_teams = set()
        for home, away in pairings:
            if matches_count[(home, away)] >= 1:
                continue
            if home in used_teams or away in used_teams:
                continue
            possible_days = [d for d in all_dates if week_start <= d < week_start + timedelta(days=7)
                             and (d - last_played[home]).days >= min_rest_days
                             and (d - last_played[away]).days >= min_rest_days
                             and day_info[d]['count'] < 2]
            if not possible_days:
                continue
            random.shuffle(possible_days)
            for date in possible_days:
                available_times = [t for t in match_times if t not in day_info[date]['times']]
                available_venues = [v for v in venues if v not in day_info[date]['venues']]
                if not available_times or not available_venues:
                    continue
                time = random.choice(available_times)
                venue = random.choice(available_venues)
                match = Match(home, away, date, time, venue)
                schedule.append(match)
                last_played[home] = date
                last_played[away] = date
                matches_count[(home, away)] += 1
                used_teams.update([home, away])
                day_info[date]['count'] += 1
                day_info[date]['venues'].add(venue)
                day_info[date]['times'].add(time)
                break

    schedule.sort(key=lambda m: m.date)
    return schedule

# طباعة جدول كل فريق
def print_team_schedules(schedule):
    team_schedule = {team: [] for team in teams}
    for match in schedule:
        team_schedule[match.team1].append(match)
        team_schedule[match.team2].append(match)
    for team, matches in team_schedule.items():
        print(f"\n=== Schedule for {team} ===")
        matches.sort(key=lambda m: m.date)
        for m in matches:
            opponent = m.team2 if m.team1 == team else m.team1
            print(f"{m.date.strftime('%Y-%m-%d')} at {m.time} vs {opponent} in {m.venue}")

# توليد عدة احتمالات
num_possibilities = 5
all_possible_schedules = [generate_weekly_schedule(teams, venues, all_dates, match_times) for _ in range(num_possibilities)]

# طباعة أول 5 احتمالات
for i, schedule in enumerate(all_possible_schedules):
    print(f"\n=== Schedule {i+1} ===")
    for m in schedule[:10]:  # أول 10 مباريات لكل جدول
        print(m)
    print_team_schedules(schedule)  # جدول كل فريق



=== Schedule 1 ===
Masry vs Future on 2025-05-01 at 20:00 in Military Academy Stadium
Zamalek vs Smouha on 2025-05-01 at 17:00 in Borg El Arab
Zed vs ENPPI on 2025-05-02 at 20:00 in Al Salam Stadium
National Bank vs Ismaily on 2025-05-02 at 17:00 in Cairo Stadium
Ceramica vs Pharco on 2025-05-05 at 17:00 in Air Defense Stadium
Alexandria Union vs El Dakhleya on 2025-05-05 at 20:00 in Borg El Arab
Wadi Degla vs Pyramids on 2025-05-06 at 20:00 in Cairo Stadium
El Gouna vs Talaea El Gaish on 2025-05-07 at 20:00 in Cairo Stadium
Al Ahly vs Modern Sport on 2025-05-07 at 17:00 in Borg El Arab
ENPPI vs Ismaily on 2025-05-08 at 20:00 in Alexandria Stadium

=== Schedule for Al Ahly ===
2025-05-07 at 17:00 vs Modern Sport in Borg El Arab
2025-05-12 at 17:00 vs Masry in Borg El Arab
2025-05-16 at 20:00 vs Alexandria Union in Suez Stadium
2025-05-23 at 17:00 vs Pyramids in El Sekka El Hadeed Stadium
2025-06-04 at 20:00 vs Ceramica in Petro Sport Stadium
2025-06-10 at 17:00 vs El Dakhleya in Alexa

In [5]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Dropdown لاختيار الاحتمال
schedule_dropdown = widgets.Dropdown(
    options=[f"Schedule {i+1}" for i in range(num_possibilities)],
    description="Choose:",
    disabled=False
)

# Dropdown لاختيار نوع العرض
view_dropdown = widgets.Dropdown(
    options=["Full League Schedule", "Team Schedule"],
    description="View:",
    disabled=False
)

# Dropdown لاختيار الفريق
team_dropdown = widgets.Dropdown(
    options=teams,
    description="Team:",
    disabled=False
)

# زرار للعرض
button = widgets.Button(description="Show Schedule")

# Output area
output = widgets.Output()

# الدالة المسؤولة عن العرض
def show_schedule(b):
    with output:
        clear_output(wait=True)  # يمسح الجدول القديم قبل العرض
        selected_index = int(schedule_dropdown.value.split()[1]) - 1
        schedule = all_possible_schedules[selected_index]
        view = view_dropdown.value
        if view == "Full League Schedule":
            html = "<h2>Full League Schedule</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Home</th><th>Away</th><th>Venue</th></tr>"
            for match in schedule:
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{match.team1}</td><td>{match.team2}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))
        else:
            team = team_dropdown.value
            team_matches = [m for m in schedule if m.team1 == team or m.team2 == team]
            html = f"<h2>Schedule for {team}</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Opponent</th><th>Venue</th></tr>"
            for match in team_matches:
                opponent = match.team2 if match.team1 == team else match.team1
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{opponent}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))

button.on_click(show_schedule)

# عرض الـ widgets
display(schedule_dropdown, view_dropdown, team_dropdown, button, output)


Dropdown(description='Choose:', options=('Schedule 1', 'Schedule 2', 'Schedule 3', 'Schedule 4', 'Schedule 5')…

Dropdown(description='View:', options=('Full League Schedule', 'Team Schedule'), value='Full League Schedule')

Dropdown(description='Team:', options=('Al Ahly', 'Zamalek', 'Pyramids', 'Masry', 'Future', 'Ismaily', 'Smouha…

Button(description='Show Schedule', style=ButtonStyle())

Output()